In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install PyMuPDF
!pip install opencv-python

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
from PIL import Image
import fitz
import re
import pandas as pd
import cv2
import pytesseract

In [ ]:
import numpy as np
name = "AK_T7_VERKOOPpdf"
pdf = fitz.open(f"{name}.pdf")
datadict = {}

#1 - Bepalen of annotaties voldoende zijn
def get_data_from_annots(pdf):
  annot_list = []
  annots = pdf[0].annots(types=fitz.PDF_ANNOT_SQUARE)
  for annot in annots:
    annot_content = annot.get_text(option="text")
    annot_list.append(annot_content) if len(annot_content)>0 else 0

  annot_list = [element.split("\n") for element in annot_list]
  annot_list = [subitem for item in annot_list for subitem in item if len(subitem)>0]

  #2 - Controleer of annotaties de aktereferentiecode bevatten
  ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
  reference_code = [element for element in annot_list if re.search(ref_regex,element)]

  if len(reference_code)<1:
    #Geen referentiecode gedetecteerd in annots
    text_page_1 = pdf[0].get_text()
    reference_code = re.search(ref_regex,text_page_1)[0]
    if len(reference_code)<1:
      return None
  return reference_code, annot_list

#3 - Extraheren uit selecteerbare tekst
def get_data_from_selectable_text(pdf, get_ref_code=False):
  if isinstance(pdf,str) == False:
    textstring = pdf[0].get_text()
  else:
    textstring = pdf
  try:
    textstring = textstring.replace(","," ").replace(".","\next").replace("?","\next").replace("\n","\next").replace('"',' ').replace("!","\next").split("\next")
  except:
    return None
  worddict={}
  if get_ref_code:
    ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
    return re.search(ref_regex, textstring)
  for word in textstring:
    if word in worddict.keys():
      worddict[word] += 1
    else:
      worddict[word] = 1
  reg_col = re.compile("([0-9]{1,}){0,}.(?:[A-Z]{3,})")
  df = pd.DataFrame.from_dict(worddict, orient="index").sort_values(by=0).T
  necessary_cols = [col for col in df.columns if re.search(reg_col,col)]
  df = df.loc[:,necessary_cols]
  df.columns = [re.search(reg_col,col)[0].strip() for col in df.columns]
  df = df.loc[:,~df.columns.duplicated()]
  return df

#4 - Via OCR-extraheren
def get_data_from_ocr(pdf, get_ref_code=False):
  textstring = ""
  max_range = pdf.page_count if get_ref_code else pdf.page_count if pdf.page_count<3 else 3
  for i in range(max_range):
    img_first_page = fitz.Pixmap(pdf, pdf.get_page_images(i)[0][0])
    img_first_page.save("temp_img_text.jpg")
    del img_first_page
    img = cv2.imread("temp_img_text.jpg")
    textstring += pytesseract.image_to_string(img) 
  if get_ref_code:
    ref_regex = re.compile("[0-9].-[A-Z]-([0-9].\/)*[0-9]{4}-[0-9]{5}")
    return re.search(ref_regex, textstring)
  text1_worddf = get_data_from_selectable_text(textstring)
  return list(text1_worddf.columns)

def get_reference_code_from_text(pdf):
  ref = get_data_from_selectable_text(pdf, True)
  if ref is None:
    None
  else:
    return ref[0]

reference_code, annot_list = get_data_from_annots(pdf)
if reference_code is None:
  reference_code = get_reference_code_from_text(pdf)
  if reference_code is None:
    datadict["Reference Code"] = "Error - not detected within document."
  else:
    datadict["Reference Code"] = reference_code
else:
  if isinstance(reference_code,str)==False:
    reference_code = reference_code[0]
  datadict["Reference Code"] = reference_code
  datadict["Annotations"] = annot_list
  text_data = get_data_from_selectable_text(pdf)
  if text_data is None or len(text_data.columns)<3:
    datadict["Extra data from text"] = get_data_from_ocr(pdf)
  else:
    datadict["Extra data from text"] = list(text_data.columns)

datadict



{'Annotations': ['4 p - 12+6=18 blz ', 'Verkoop € 110.000 ', 'ontslag'],
 'Extra data from text': ['OORSPRONG',
  'ONDERTEKENING',
  'VERKOOP',
  'STAD',
  'VERKOOPSVOORWAARDEN',
  'DERUYTERE',
  'SOETE',
  'DEVAERE',
  '“BOES',
  'LAM',
  'BRECHT',
  'VANDENDRIESSCHE',
  'BTW',
  'SERVAYSE',
  'VANACKER',
  'DEMEYERE'],
 'Reference Code': '64-T-19/11/2020-08234'}